In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
import sys, urllib, json
import urllib.request
from sklearn.preprocessing import *

%matplotlib inline

In [2]:
# 导入数据
df = pd.read_csv('train_20171215.txt', sep='\t')

print(df.info())
print(df.describe())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 4 columns):
date           4773 non-null int64
day_of_week    4773 non-null int64
brand          4773 non-null int64
cnt            4773 non-null int64
dtypes: int64(4)
memory usage: 149.2 KB
None
              date  day_of_week        brand          cnt
count  4773.000000  4773.000000  4773.000000  4773.000000
mean    524.399749     3.512047     2.990153   380.567358
std     297.546919     1.738846     1.417509   252.720918
min       1.000000     1.000000     1.000000    12.000000
25%     271.000000     2.000000     2.000000   221.000000
50%     529.000000     3.000000     3.000000   351.000000
75%     781.000000     5.000000     4.000000   496.000000
max    1032.000000     7.000000     5.000000  2102.000000
   date  day_of_week  brand   cnt
0     1            3      1    20
1     1            3      5    48
2     2            4      1    16
3     2            4      3    20
4     3          

In [3]:
# 标记周数
def tickWeek(df, start):
    preVal = df[:-1]['day_of_week'].values
    preVal = np.insert(preVal, 0, preVal[0]-1)
    df['new_week'] = (preVal >= df.day_of_week)        #判断是否是新的一周
    weekList = list()
    week = start
    for item in df['new_week']:
        week += (1 if item else 0)
        weekList.append(week)
    df['week'] = weekList
    return df

In [4]:
# 标记周数+修正日期
dataDf = pd.pivot_table(df,index=["date"], values=["cnt","day_of_week"], aggfunc={"cnt":np.sum, "day_of_week": np.max})
dataDf = tickWeek(dataDf, 0)
dataDf['day'] = dataDf['week']*7 + dataDf['day_of_week']
print(dataDf.head(10))

       cnt  day_of_week  new_week  week  day
date                                        
1       68            3     False     0    3
2       36            4     False     0    4
3     5565            5     False     0    5
4     4966            6     False     0    6
5     3346            7     False     0    7
6     3396            1      True     1    8
7     4146            2     False     1    9
8     3096            3     False     1   10
9     2713            4     False     1   11
10    2409            5     False     1   12


In [5]:
# 给数据添加日期字段
def addGuessDate(df, startYear, startMonth, startDay):
    startDate = date(startYear, startMonth, startDay)
    df['guess_date'] = df['day'].map(lambda x: startDate + timedelta(days=x))
    df['guess_date'] = pd.to_datetime(df['guess_date'])
    return df

In [6]:
# 尝试给数据加日期
dataDf = addGuessDate(dataDf,2012,12,30)
dataDf = dataDf.drop('new_week', 1)

print(dataDf.head(20))

       cnt  day_of_week  week  day guess_date
date                                         
1       68            3     0    3 2013-01-02
2       36            4     0    4 2013-01-03
3     5565            5     0    5 2013-01-04
4     4966            6     0    6 2013-01-05
5     3346            7     0    7 2013-01-06
6     3396            1     1    8 2013-01-07
7     4146            2     1    9 2013-01-08
8     3096            3     1   10 2013-01-09
9     2713            4     1   11 2013-01-10
10    2409            5     1   12 2013-01-11
11     275            6     1   13 2013-01-12
12    2735            1     2   15 2013-01-14
13    2386            2     2   16 2013-01-15
14    1901            3     2   17 2013-01-16
15    6072            4     2   18 2013-01-17
16    2716            5     2   19 2013-01-18
17     216            6     2   20 2013-01-19
18      22            7     2   21 2013-01-20
19    3903            1     3   22 2013-01-21
20    2700            2     3   23

In [7]:
# 给数据集添加前一个日期的列
preDate = dataDf[:-1]['guess_date'].values
preDate = np.insert(preDate, 0, datetime(2013, 1, 1))
dataDf['pre_date'] = preDate
print(dataDf.head(10))

       cnt  day_of_week  week  day guess_date   pre_date
date                                                    
1       68            3     0    3 2013-01-02 2013-01-01
2       36            4     0    4 2013-01-03 2013-01-02
3     5565            5     0    5 2013-01-04 2013-01-03
4     4966            6     0    6 2013-01-05 2013-01-04
5     3346            7     0    7 2013-01-06 2013-01-05
6     3396            1     1    8 2013-01-07 2013-01-06
7     4146            2     1    9 2013-01-08 2013-01-07
8     3096            3     1   10 2013-01-09 2013-01-08
9     2713            4     1   11 2013-01-10 2013-01-09
10    2409            5     1   12 2013-01-11 2013-01-10


In [8]:
# 尝试得到缺失的日期数据
fillingDf = dataDf[:0][['cnt', 'day_of_week', 'week', 'guess_date']]
print(fillingDf.head(10))

Empty DataFrame
Columns: [cnt, day_of_week, week, guess_date]
Index: []


In [9]:
dataDf['day_diff'] = (dataDf['guess_date'] - dataDf['pre_date']).map(lambda x : x.days)
# guess_date = dataDf.iloc[[i], [4]].values
# print(guess_date[0][0] - 10 * 24 * 60 * 60 * 1000000000)

# 获取缺失日期的dataFrame
for i in range(len(dataDf)):
    cnt = dataDf.iloc[[i], [0]].values
    day_of_week = dataDf.iloc[[i], [1]].values
    week = dataDf.iloc[[i], [2]].values
    day = dataDf.iloc[[i], [3]].values
    guess_date = dataDf.iloc[[i], [4]].values
    pre_date = dataDf.iloc[[i], [5]].values
    delta_day = dataDf.iloc[[i], [6]].values[0][0]
    if delta_day > 1:
        for n in range(delta_day - 1):
            fill_date = (guess_date - (n + 1) * 24 * 60 * 60 * 1000000000)[0][0]
            fill_day_of_week = (day_of_week - (n + 1))[0][0]
            fill_week = week[0][0]
            if fill_day_of_week < 1:
                fill_week = (int(abs(fill_day_of_week) / 7) - 1 + week)[0][0]
                fill_day_of_week = (fill_day_of_week % 7)
                if fill_day_of_week == 0:
                    fill_day_of_week = 7
            fill_cnt = 0
            fill_row = pd.DataFrame([[fill_cnt, fill_day_of_week, fill_week, fill_date]], columns=['cnt', 'day_of_week', 'week', 'guess_date'])
            fillingDf = fillingDf.append(fill_row)
print(fillingDf.head(10))

   cnt  day_of_week  week guess_date
0    0            7     1 2013-01-13
0    0            7     3 2013-01-27
0    0            3     6 2013-02-13
0    0            2     6 2013-02-12
0    0            1     6 2013-02-11
0    0            7     5 2013-02-10
0    0            6     5 2013-02-09
0    0            7     7 2013-02-24
0    0            7     8 2013-03-03
0    0            7    10 2013-03-17


In [10]:
# 尝试将原本训练集和缺失日期训练集合并
full_train_df = pd.concat([dataDf[:][['cnt', 'day_of_week', 'week', 'guess_date']], fillingDf[:][['cnt', 'day_of_week', 'week', 'guess_date']]])
print(full_train_df.tail(10))

   cnt  day_of_week  week guess_date
0    0            6   162 2016-02-13
0    0            5   162 2016-02-12
0    0            7   163 2016-02-21
0    0            7   164 2016-02-28
0    0            7   165 2016-03-06
0    0            7   166 2016-03-13
0    0            7   167 2016-03-20
0    0            7   168 2016-03-27
0    0            1   170 2016-04-04
0    0            7   169 2016-04-03


In [11]:
#给数据添加年份字段
def getDateYear(df, colName):
    df['date_year'] = df[colName].map(lambda x: x.year)
    return df

#给数据添加月份字段
def getDateMonth(df, colName):
    df['date_month'] = df[colName].map(lambda x: x.month)
    return df

full_train_df = getDateYear(full_train_df, 'guess_date')
full_train_df = getDateMonth(full_train_df, 'guess_date')
print(full_train_df.head(10))

     cnt  day_of_week  week guess_date  date_year  date_month
1     68            3     0 2013-01-02       2013           1
2     36            4     0 2013-01-03       2013           1
3   5565            5     0 2013-01-04       2013           1
4   4966            6     0 2013-01-05       2013           1
5   3346            7     0 2013-01-06       2013           1
6   3396            1     1 2013-01-07       2013           1
7   4146            2     1 2013-01-08       2013           1
8   3096            3     1 2013-01-09       2013           1
9   2713            4     1 2013-01-10       2013           1
10  2409            5     1 2013-01-11       2013           1


In [12]:
# #尝试利用第三方接口为数据添加是否工作日，假期，休息日的字段
# def get_day_type(query):
#         """
#         @query a single date: string eg."20160404"
#         @return day_type: 0 workday -1 holiday

#         20161001:2 20161002:2 20161003:2 20161004:1
#         """

#         url = 'http://tool.bitefu.net/jiari/?d=' + query
#         req = urllib.request.Request(url)
#         resp = urllib.request.urlopen(req)
#         content = resp.read().decode()

#         if(content):
#             # "0"workday, "1"leave, "2"holiday
#             if '0' in content:
#                 return 0
#             else:
#                 if '1' in content:
#                     return 1
#                 else:
#                     return 2

# def getDateProperty(df, colName):
#     df['date_property'] = df[colName].map(lambda x: get_day_type(x.strftime('%Y%m%d')))
#     return df

# holiday_df = pd.read_csv('holiday.csv')
# print(holiday_df.head(10))

# 尝试利用holiday文件获取日期是否节假日特征
def get_day_type(df, colName):
    holiday_df = pd.read_csv('holiday.csv')
    df['date_property'] = 0
    df['guess_date_str'] = df[colName].map(lambda x: x.strftime('%Y-%m-%d'))
#     holiday_df['guess_date_str'] = holiday_df['date'].map(lambda x: x.strftime('%Y-%m-%d'))
    for guess_date_str in df['guess_date_str']:
#         print(guess_date)
#         print(holiday_df.loc[0]['date'])
        df.date_property[df.guess_date_str == guess_date_str] = holiday_df['holiday_type'][holiday_df.date == guess_date_str]
    return df

full_train_df = get_day_type(full_train_df, 'guess_date')
print(full_train_df.head(10))

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  
1      2013-01-02  
2      2013-01-03  
3      2013-01-04  
4      2013-01-05  
5      2013-01-06  

In [13]:
#将数据集按日期升序重新排序
full_train_df = full_train_df.sort_values(by='guess_date_str', ascending=True)
print(full_train_df.head(50))

     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   
11   275            6     1 2013-01-12       2013           1              1   
0      0            7     1 2013-01-13  

In [14]:
#尝试使用另一个比赛的销售量数据作为特征
def getSaleQuantity(sale_quantity_data, date):
    sale_date_this_month = date.year * 100 + date.month
    if date.month == 1:
        sale_date_last_month = (date.year - 1) * 100 + 12
    else:
        sale_date_last_month = sale_date_this_month - 1
    sale_quantity_this_month = sale_quantity_data.sale_quantity[sale_quantity_data.sale_date == sale_date_this_month].sum()
    sale_quantity_last_month = sale_quantity_data.sale_quantity[sale_quantity_data.sale_date == sale_date_last_month].sum()
    result = (sale_quantity_this_month + sale_quantity_last_month) / 2
    return result

#对数据集填充销售量数据
def fillSaleQuantity(df, sale_quantity_data):
    df['sale_quantity'] = 0
    for date in df['guess_date']:
        df.sale_quantity[(df.date_year == date.year) & (df.date_month == date.month)] = getSaleQuantity(sale_quantity_data, date)
    return df

sale_quantity_data = pd.read_csv('yancheng_train_20171226.csv')

#给训练集添加sale_quantity字段
full_train_df = fillSaleQuantity(full_train_df, sale_quantity_data)
print(full_train_df.head(10))

D:\win10-Application\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity  
1      2013-01-02        28137.0  
2      2013-01-03        28137.0  
3      2013-01

In [15]:
#给数据添加属于当月上旬还是下旬特征数据
def addDividedMonth(df, colName):
    df[colName] = df.guess_date.map(lambda x: 'early' if (x.day <= 15) else 'late')
    return df

full_train_df = addDividedMonth(full_train_df, 'dividedMonth')
print(full_train_df.tail(10))

       cnt  day_of_week  week guess_date  date_year  date_month  \
1025  3208            2   169 2016-03-29       2016           3   
1026  2277            3   169 2016-03-30       2016           3   
1027  2144            4   169 2016-03-31       2016           3   
1028  2519            5   169 2016-04-01       2016           4   
1029   195            6   169 2016-04-02       2016           4   
0        0            7   169 2016-04-03       2016           4   
0        0            1   170 2016-04-04       2016           4   
1030  4003            2   170 2016-04-05       2016           4   
1031  2513            3   170 2016-04-06       2016           4   
1032  1306            4   170 2016-04-07       2016           4   

      date_property guess_date_str  sale_quantity dividedMonth  
1025              0     2016-03-29        41868.5         late  
1026              0     2016-03-30        41868.5         late  
1027              0     2016-03-31        41868.5         late  
10

In [16]:
#给数据集添加是否是休息日或节假日后一天特征
def addRestdayAfterOneDay(df, colName, insert):
    preVal = df[:-1]['date_property'].values
    preVal = np.insert(preVal, 0, insert)
    df['pre_date_property'] = preVal
    df[colName] = 0
    df[colName][(df.date_property == 0) & ((df.pre_date_property == 2) | (df.pre_date_property == 1))] = 1
    return df

full_train_df = addRestdayAfterOneDay(full_train_df, 'after_restday_one', 2)
print(full_train_df.head(10))

     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity dividedMonth  pre_date_property  \
1      2013-01-02        28137.0        early      

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [17]:
# #给数据集添加是否是节假日后一天特征
# def addHolidayAfterOneDay(df, colName, index):
#     df[colName] = 0
#     length = len(df[colName])
#     for i in range(length):
#         if df.loc[(i + index)]['date_property'] == 2:
#             for j in range(30):
#                 if (i + j) < length:
#                     if df.loc[(i + j + index)]['date_property'] == 0:
#                         df[colName].loc[(i + j + index)] = 1
#                         break
#                 else:
#                     break
#         else:
#             continue
#     return df

#给数据集手动添加是否是节假日后一天特征
def addHolidayAfterOneDayByDate(df, colName, date_set):
    df[colName] = 0
    date_set = pd.to_datetime(date_set)
    for date in date_set:
        df[colName][df.guess_date == date] = 1
    return df

# full_train_df = addHolidayAfterOneDay(full_train_df, 'after_holiday_one', 1)
train_date_set = ['2013-01-04', '2013-02-16', '2013-04-07', '2013-05-02', '2013-06-13', '2013-09-22', '2013-10-08',
                 '2014-01-02', '2014-02-07', '2014-04-08', '2014-05-04', '2014-06-03', '2014-09-09', '2014-10-08',
                 '2015-01-04', '2015-02-25', '2015-04-07', '2015-05-04', '2015-06-23', '2015-09-06', '2015-09-28', '2015-10-08',
                 '2016-01-04', '2016-02-14', '2016-04-05']
full_train_df = addHolidayAfterOneDayByDate(full_train_df, 'after_holiday_one', train_date_set)

print(full_train_df.head(10))

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity dividedMonth  pre_date_property  \
1      2013-01-02        28137.0        early      

In [18]:
#给数据集添加是否是节假轮休日特征，即day_of_week等于1-5，date_property等于1或者2的日期
def addIsHolidayRestDay(df, colName):
    df[colName] = 0
    df[colName][((df.date_property == 1) | (df.date_property == 2)) & (df.day_of_week <= 5)] = 1
    return df

full_train_df = addIsHolidayRestDay(full_train_df, 'is_holi_restday')
print(full_train_df.head(10))

     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity dividedMonth  pre_date_property  \
1      2013-01-02        28137.0        early      

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
#给数据集手动添加是否是元旦假期特征
def addIsNewYearDay(df, colName, date_set):
    df[colName] = 0
    date_set = pd.to_datetime(date_set)
    for date in date_set:
        df[colName][df.guess_date == date] = 1
    return df

train_newYearDay_set = ['2013-01-02', '2013-01-03', '2014-01-01', '2015-01-01', '2015-01-02', '2015-01-03', '2016-01-01', '2016-01-02', '2016-01-03']
full_train_df = addIsNewYearDay(full_train_df, 'is_newYearDay', train_newYearDay_set)

print(full_train_df.head(10))

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity dividedMonth  pre_date_property  \
1      2013-01-02        28137.0        early      

In [20]:
#给数据集添加是否是节假周末的特征
def addIsHoliWeekend(df, colName, after_date_property_insert, pre_isHoliWeekend_insert, after_isHoliWeekend_insert):
    df[colName] = 0
    df[colName][(df.day_of_week == 6) & ((df.date_property == 2) | ((df.date_property == 1) & (df.pre_date_property == 2)))] = 1
    afterVal = df['date_property'].values
    afterVal = afterVal[1:]
    afterVal = np.insert(afterVal, -1, after_date_property_insert)
    df['after_date_property'] = afterVal
    df[colName][(df.day_of_week == 7) & ((df.date_property == 2) | ((df.date_property == 1) & (df.after_date_property == 2)))] = 1
    
    pre_isHolidayWeekend = df[colName].values
    pre_isHolidayWeekend = pre_isHolidayWeekend[:(- 1)]
    pre_isHolidayWeekend = np.insert(pre_isHolidayWeekend, 0, pre_isHoliWeekend_insert)
    df['pre_isHolidayWeekend'] = pre_isHolidayWeekend
    
    after_isHolidayWeekend = df[colName].values
    after_isHolidayWeekend = after_isHolidayWeekend[1:]
    after_isHolidayWeekend = np.insert(after_isHolidayWeekend, -1, after_isHoliWeekend_insert)
    df['after_isHolidayWeekend'] = after_isHolidayWeekend
    
    df[colName][(df.day_of_week == 6) & (df.after_isHolidayWeekend == 1) & (df.date_property == 1)] = 1
    df[colName][(df.day_of_week == 7) & (df.pre_isHolidayWeekend == 1) & (df.date_property == 1)] = 1
    return df

full_train_df = addIsHoliWeekend(full_train_df, 'isHolidayWeekend', 0, 0, 0)
print(full_train_df.head(10))

     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity dividedMonth  pre_date_property  \
1      2013-01-02        28137.0        early      

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py

In [21]:
#给数据集添加是否是纯休周末特征
def addIsPureWeekend(df, colName):
    df[colName] = 0
    df[colName][(df.date_property == 1) & (df.isHolidayWeekend == 0) & ((df.day_of_week == 6) | (df.day_of_week == 7))] = 1
    return df

full_train_df = addIsPureWeekend(full_train_df, 'isPureWeekend')
print(full_train_df.head(10))

     cnt  day_of_week  week guess_date  date_year  date_month  date_property  \
1     68            3     0 2013-01-02       2013           1              2   
2     36            4     0 2013-01-03       2013           1              2   
3   5565            5     0 2013-01-04       2013           1              0   
4   4966            6     0 2013-01-05       2013           1              0   
5   3346            7     0 2013-01-06       2013           1              0   
6   3396            1     1 2013-01-07       2013           1              0   
7   4146            2     1 2013-01-08       2013           1              0   
8   3096            3     1 2013-01-09       2013           1              0   
9   2713            4     1 2013-01-10       2013           1              0   
10  2409            5     1 2013-01-11       2013           1              0   

   guess_date_str  sale_quantity dividedMonth  pre_date_property  \
1      2013-01-02        28137.0        early      

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [31]:
#处理测试集A相关数据
df_test_A = pd.read_csv('test_A_20171225.txt', sep='\t')

df_test_A.set_index('date', inplace=True)
df_test_A = tickWeek(df_test_A, dataDf.loc[dataDf.index[-1], 'week'])
df_test_A['day'] = df_test_A['week'] * 7 + df_test_A['day_of_week']

df_test_A = addGuessDate(df_test_A,2012,12,30)
df_test_A = df_test_A.drop('new_week', 1)

df_test_A = getDateYear(df_test_A, 'guess_date')
df_test_A = getDateMonth(df_test_A, 'guess_date')

# 尝试利用holiday文件获取日期是否节假日特征
def get_test_day_type(df, colName):
    holiday_df = pd.read_csv('holiday.csv')
    df['date_property'] = 0
    df['guess_date_str'] = df[colName].map(lambda x: x.strftime('%Y-%m-%d'))
#     holiday_df['guess_date_str'] = holiday_df['date'].map(lambda x: x.strftime('%Y-%m-%d'))
    for guess_date_str in df['guess_date_str']:
#         print(holiday_df.holiday_type[holiday_df.date == guess_date_str])
        df.date_property[df.guess_date_str == guess_date_str] = holiday_df.holiday_type[holiday_df.date == guess_date_str].values
    return df

df_test_A = get_test_day_type(df_test_A, 'guess_date')

df_test_A = fillSaleQuantity(df_test_A, sale_quantity_data)

df_test_A = addDividedMonth(df_test_A, 'dividedMonth')

df_test_A = addRestdayAfterOneDay(df_test_A, 'after_restday_one', 0)

test_date_set = ['2016-05-03', '2016-06-12', '2016-09-18', '2016-10-08', '2017-01-03', '2017-02-03']
df_test_A = addHolidayAfterOneDayByDate(df_test_A, 'after_holiday_one', test_date_set)

df_test_A = addIsHolidayRestDay(df_test_A, 'is_holi_restday')

test_newYearDay_set = ['2017-01-01', '2017-01-02']
df_test_A = addIsNewYearDay(df_test_A, 'is_newYearDay', test_newYearDay_set)

df_test_A = addIsHoliWeekend(df_test_A, 'isHolidayWeekend', 0, 0, 0)
df_test_A = addIsPureWeekend(df_test_A, 'isPureWeekend')

print(df_test_A.head(10))

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be s

      day_of_week  week   day guess_date  date_year  date_month  \
date                                                              
1032            4   170  1194 2016-04-07       2016           4   
1033            5   170  1195 2016-04-08       2016           4   
1034            6   170  1196 2016-04-09       2016           4   
1035            1   171  1198 2016-04-11       2016           4   
1036            2   171  1199 2016-04-12       2016           4   
1037            3   171  1200 2016-04-13       2016           4   
1038            4   171  1201 2016-04-14       2016           4   
1039            5   171  1202 2016-04-15       2016           4   
1040            6   171  1203 2016-04-16       2016           4   
1041            1   172  1205 2016-04-18       2016           4   

      date_property guess_date_str  sale_quantity dividedMonth  \
date                                                             
1032              0     2016-04-07        41163.0        early 

D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\win10-Application\Anaconda\lib\site-packages\ipykernel_launcher.

In [32]:
# 导出训练集预处理结果
def exportResult(df, fileName):
    df.to_csv('./%s.csv' % fileName, header=True, index=False)
    
#缩放字段至0-1
def scalerFea(df, cols):
    df.dropna(inplace=True)
    scaler = MinMaxScaler()
    df[cols] = scaler.fit_transform(df[cols].values)
    return df,scaler.scale_

#将相关字段进行归一化处理
def scaleData(train_df, test_df, colSet):
    df = pd.concat([train_df[colSet], test_df[colSet]], axis=0)
    for colName in colSet:
        scaler = MinMaxScaler()
        df[colName] = scaler.fit_transform(df[colName].values)
        train_df[colName + '_scaled'] = df[:len(train_df)][colName]
        test_df[colName + '_scaled'] = df[len(train_df):][colName]
    return train_df, test_df

scaleColSet = ['week', 'date_year', 'sale_quantity']
full_train_df, test_df = scaleData(full_train_df, df_test_A, scaleColSet)
print(test_df.head(10))

      day_of_week  week   day guess_date  date_year  date_month  \
date                                                              
1032            4   170  1194 2016-04-07       2016           4   
1033            5   170  1195 2016-04-08       2016           4   
1034            6   170  1196 2016-04-09       2016           4   
1035            1   171  1198 2016-04-11       2016           4   
1036            2   171  1199 2016-04-12       2016           4   
1037            3   171  1200 2016-04-13       2016           4   
1038            4   171  1201 2016-04-14       2016           4   
1039            5   171  1202 2016-04-15       2016           4   
1040            6   171  1203 2016-04-16       2016           4   
1041            1   172  1205 2016-04-18       2016           4   

      date_property guess_date_str  sale_quantity dividedMonth  \
date                                                             
1032              0     2016-04-07        41163.0        early 

D:\win10-Application\Anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
D:\win10-Application\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
D:\win10-Application\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
D:\win10-Application\Anaconda

In [33]:
#添加one-hot编码并保留原字段
def addOneHot(df, colName):
    colDum = pd.get_dummies(df[colName], prefix=colName)
    df = pd.concat([df, colDum], axis=1)
    return df

#将相关字段进行one-hot编码
def oneHotData(train_df, test_df, colSet):
    for colName in colSet:
        train_df = addOneHot(train_df, colName)
        test_df = addOneHot(test_df, colName)
    return train_df, test_df

oneHotColSet = ['day_of_week', 'date_property', 'date_month', 'dividedMonth']
full_train_df, test_df = oneHotData(full_train_df, test_df, oneHotColSet)

#因为预测的数据集中3月份为空，因此手动赋值date_month_3
test_df['date_month_3'] = 0

print(test_df.head(10))

      day_of_week  week   day guess_date  date_year  date_month  \
date                                                              
1032            4   170  1194 2016-04-07       2016           4   
1033            5   170  1195 2016-04-08       2016           4   
1034            6   170  1196 2016-04-09       2016           4   
1035            1   171  1198 2016-04-11       2016           4   
1036            2   171  1199 2016-04-12       2016           4   
1037            3   171  1200 2016-04-13       2016           4   
1038            4   171  1201 2016-04-14       2016           4   
1039            5   171  1202 2016-04-15       2016           4   
1040            6   171  1203 2016-04-16       2016           4   
1041            1   172  1205 2016-04-18       2016           4   

      date_property guess_date_str  sale_quantity dividedMonth      ...       \
date                                                                ...        
1032              0     2016-04-07 

In [34]:
#导出训练集
exportResult(full_train_df, 'A_fulltraining_feature_set')

#导出测试集A
exportResult(test_df, 'A_fulltesting_feature_set')